# scan different paramters for the first low res registartion

In [22]:
%load_ext autoreload
%autoreload 2
import sys
from tifffile import imwrite as tif_imwrite
from tifffile import imread as tif_imread
from IPython.display import display, HTML
import numpy as np
import os
import hjson
from pathlib import Path
import os
sys.path.append("../../")
from registrations_funcs import HCR_confocal_imaging

display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
import socket

# Path for bigstream unless you did pip install
sys.path = [fr"\\nasquatch\data\2p\jonna\Code_Python\Notebooks_Jonna\BigStream\bigstream_github"] + sys.path 
sys.path = [fr"C:\Users\jonna\Notebooks_Jonna\BigStream\bigstream_github"] + sys.path 
sys.path = [fr'{os.getcwd()}/bigstream_github'] + sys.path



manifest = hjson.load(open(hjson_file, 'r'))

# verify that all rounds exists.
reference_round_path, mov_rounds_path = HCR_confocal_imaging(manifest, only_paths=True)
reference_round_number = manifest['HCR_confocal_imaging']['reference_round']
print("Rounds available for register:")
round_to_rounds = {}
j=0
for i in manifest['HCR_confocal_imaging']['rounds']:
    if i['round'] != reference_round_number:
        print(i['round'],i['channels'])
        round_to_rounds[i['round']] = i
        round_to_rounds[i['round']]['image_path'] = mov_rounds_path[j]
        j+=1
    else:
        reference_round = i
        reference_round['image_path'] = reference_round_path



hjson_file = fr'/home/oamsalem/code/Projects/EZfish/ezfish_pipeline/examples/RG026.hjson'



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


All files found
Rounds available for register:
02 ['DAPI', 'GLP1R', 'CHAT', 'TAC1']
03 ['DAPI', 'RORB', 'NPR3', 'VGAT']


# PARAMETERS

In [19]:
HCR_round_to_register = '02'# 02,03...?
assert HCR_round_to_register in round_to_rounds.keys(), f"round {HCR_round_to_register} not found in rounds to register"



### Image parameters

In [20]:
mov_round = round_to_rounds[HCR_round_to_register]
# File names
HCR_fix_image_path = reference_round['image_path'] # The fix image that all other rounds will be registerd to (include all channels!)
HCR_mov_image_path = mov_round['image_path'] # The image that will be registered to the fix image (include all channels!)

output_folder_name = 'output_HCR5_to_HCR1_github'

# resolution of the images
fix_image_spacing = np.array(reference_round['resolution']) # Y,X,Z
mov_image_spacing = np.array(mov_round['resolution']) # Y,X,Z

# spatial downsampling, probably no need to test. (Changed x and y from 3 to 2 for CIM round 5)
red_mut_x = 2 
red_mut_y = 2
red_mut_z = 2


### bigstream global parameters

In [54]:
# radius for local ransac for each spot (in pixles for LOW res image) - how many pixels to look around a spot for match
g_radii_z = [1,5,15] #Typical values: 1,3,5
g_radii_xy = [5, 10, 25, 35] #Typical values: 5, 8, 10

# when searching for blobs for ransac, what is the radius for the search alogorithem (in pixles for low res image)
g_bsize_s_b = [[3,10], [3,8], [5,15], [5, 20], [2,30],[4,30]]

# what is the minumum correlation for a spot to be considered a match
g_m_tresholds = [0.1, 0.2, 0.3, 0.4] 


#### Load the images

In [21]:
# Take first channel, I always assume that the first channel is what we want to register
fix_highres = tif_imread(HCR_fix_image_path)[:,0].transpose(2,1,0) # take first channlel and transform from Z,X,Y to Y,X,Z
mov_highres = tif_imread(HCR_mov_image_path)[:,0].transpose(2,1,0) # take first channlel and transform from Z,X,Y to Y,X,Z


fix_lowres = fix_highres[::red_mut_y,::red_mut_x,::red_mut_z]
fix_lowres_spacing = fix_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])


mov_lowres = mov_highres[::red_mut_y,::red_mut_x,::red_mut_z]
mov_lowres_spacing = mov_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])

# Print the dimensions of the high-resolution and low-resolution images
print(f"The dimensions of the high-resolution fixed image are: {fix_highres.shape}")
print(f"The dimensions of the high-resolution moving image are: {mov_highres.shape}")
print(f"The dimensions of the low-resolution moving image are: {mov_lowres.shape}")

The dimensions of the high-resolution fixed image are: (2922, 2914, 71)
The dimensions of the high-resolution moving image are: (3437, 3555, 77)
The dimensions of the low-resolution moving image are: (1719, 1778, 39)


In [56]:
success_n = 0
write_dir = base_path / output_folder_name / 'global_parms_scan'
write_dir.mkdir(exist_ok=True,parents=True)
write_directory = str(write_dir)
print(write_directory)
# saving 1 version of the mov lowres
tif_imwrite(f'{write_directory}/mov_lowres.tiff', mov_lowres.transpose(2,1,0))

for g_radius_z in g_radii_z:
  for g_radius_xy in g_radii_xy:
    for g_bsize_s, g_bsize_b in g_bsize_s_b:
      for g_m_treshold in g_m_tresholds:
        
        fname = f"rz{g_radius_z}_rxy{g_radius_xy}_bs{g_bsize_s}to{g_bsize_b}_mt{g_m_treshold}"

        global_ransac_kwargs = {'cc_radius':(g_radius_xy,g_radius_xy,g_radius_z),
                                'blob_sizes':[g_bsize_s, g_bsize_b],
                                'match_threshold':g_m_treshold}

        output = register_lowres(
            fix_lowres, mov_lowres,
            fix_lowres_spacing,
            mov_lowres_spacing,
            global_ransac_kwargs=global_ransac_kwargs,
            write_directory=write_directory,
            fname=fname
        )
        if output is not None:
            success_n+=1
            
print(f' got {success_n} registration to work!')

\\bananas\data\ezfish\jonna\CIM132\HCR\240408_CIM132\output_HCR5_to_HCR1_github\global_parms_scan
1, blob_min=12 , blob_max=48
Thu Aug  8 16:39:35 2024 computing fixed spots
Thu Aug  8 16:39:35 2024 fixed spots detection using {'num_sigma': 7, 'exclude_border': (5, 5, 1)}
Thu Aug  8 16:39:35 2024 Start spot detection ((3, 3, 3), (10, 10, 10)) {'exclude_border': (5, 5, 1), 'num_sigma': 7, 'min_sigma': array([1.73205081, 1.73205081, 1.73205081]), 'max_sigma': array([5.77350269, 5.77350269, 5.77350269]), 'threshold': None, 'threshold_rel': 0.1}
Thu Aug  8 16:39:42 2024 Spot detection ((3, 3, 3), (10, 10, 10)) completed in 6.863063335418701s
Thu Aug  8 16:39:42 2024 found 3228 fixed spots
Thu Aug  8 16:39:42 2024 computing moving spots
Thu Aug  8 16:39:42 2024 moving spots detection using {'num_sigma': 7, 'exclude_border': (5, 5, 1)}
Thu Aug  8 16:39:42 2024 Start spot detection ((3, 3, 3), (10, 10, 10)) {'exclude_border': (5, 5, 1), 'num_sigma': 7, 'min_sigma': array([1.73205081, 1.732050

In [7]:
output = register_lowres(
    fix_lowres, mov_lowres,
    fix_lowres_spacing,
    mov_lowres_spacing,
    global_ransac_kwargs=global_ransac_kwargs,
    write_directory=write_directory,
    fname=fname
)

1, blob_min=18 , blob_max=73
Fri Jun 21 14:36:29 2024 computing fixed spots
Fri Jun 21 14:36:29 2024 fixed spots detection using {'num_sigma': 15, 'exclude_border': (5, 5, 1)}
Fri Jun 21 14:36:29 2024 Start spot detection ((3, 3, 3), (60, 60, 60)) {'exclude_border': (5, 5, 1), 'num_sigma': 15, 'min_sigma': array([1.73205081, 1.73205081, 1.73205081]), 'max_sigma': array([34.64101615, 34.64101615, 34.64101615]), 'threshold': None, 'threshold_rel': 0.1}
Fri Jun 21 14:36:37 2024 Spot detection ((3, 3, 3), (60, 60, 60)) completed in 8.324198246002197s
Fri Jun 21 14:36:37 2024 found 708 fixed spots
Fri Jun 21 14:36:37 2024 computing moving spots
Fri Jun 21 14:36:37 2024 moving spots detection using {'num_sigma': 15, 'exclude_border': (5, 5, 1)}
Fri Jun 21 14:36:37 2024 Start spot detection ((3, 3, 3), (60, 60, 60)) {'exclude_border': (5, 5, 1), 'num_sigma': 15, 'min_sigma': array([1.73205081, 1.73205081, 1.73205081]), 'max_sigma': array([34.64101615, 34.64101615, 34.64101615]), 'threshold': 